In [1]:
input_file = "data/gpmh.txt"

import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow_probability as tfp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

# TODO: Change root path
root = "/Users/jay/Desktop/Bachelorarbeit"

import sys

sys.path.append(f"{root}/Implementation")
from dependencies.hbv_sask.model import HBVSASKModel as hbvmodel
from src.execute_model import run_model_single_parameter_node
from src.construct_model import get_model


ndims = 7
dims = ["TT", "C0", "beta", "ETF", "FC", "FRAC", "K2"]
colors = sns.color_palette(n_colors=ndims)

testConfigPath = f"{root}/test_config.json"
with open(testConfigPath, "r") as file:
    run_config = json.load(file)

configPath = run_config["configPath"]
basis = run_config["basis"]
model = get_model(configPath, basis)

start_date: 2004-01-01 00:00:00
start_date_predictions: 2005-01-01 00:00:00
end_date: 2006-01-01 00:00:00
simulation length: 365
full_data_range is 732 hours including spin_up_length of 366 hours
simulation_range is of length 366 hours


In [2]:
# Construct params
configurationObject = model.configurationObject
param_lower = []
param_upper = []
for param in configurationObject["parameters"]:
    if param["distribution"] == "Uniform":
        param_lower.append(param["lower"])
        param_upper.append(param["upper"])
    else:
        raise NotImplementedError(
            f"Sorry, the distribution {param['distribution']} is not supported yet"
        )
param_lower = np.array(param_lower)
param_upper = np.array(param_upper)

In [3]:
samples = pd.read_csv(f"{input_file}")
samples

,TT,C0,beta,ETF,FC,FRAC,K2
0,0.911258,1.356576,2.132870,0.235092,291.572193,0.215022,0.043257
1,0.911258,1.356576,2.132870,0.235092,291.572193,0.215022,0.043257
2,-0.233729,0.588043,2.198162,0.113685,233.553188,0.328521,0.043458
3,-0.233729,0.588043,2.198162,0.113685,233.553188,0.328521,0.043458
4,-0.233729,0.588043,2.198162,0.113685,233.553188,0.328521,0.043458
...,...,...,...,...,...,...,...
884,1.951661,6.653201,1.385673,0.099440,209.911936,0.171404,0.025641
885,2.792069,5.376126,1.751294,0.129530,115.271779,0.137601,0.041602
886,3.831516,5.807979,2.020655,0.008888,157.787768,0.276033,0.042169
887,3.831516,5.807979,2.020655,0.008888,157.787768,0.276033,0.042169


# Plotting

In [4]:
fig = make_subplots(rows=2, cols=4)

# Histogram and KDE
for i, col in enumerate(samples.columns):
    row = (i // 4) + 1
    col_idx = (i % 4) + 1

    # Histogram
    fig.add_trace(
        go.Histogram(x=samples[col], name=col, histnorm="probability density"),
        row=row,
        col=col_idx,
    )

    # KDE calculation
    kde = gaussian_kde(samples[col])
    x_values = np.linspace(samples[col].min(), samples[col].max(), 300)
    kde_values = kde(x_values)
    fig.add_trace(
        go.Scatter(
            x=x_values, y=kde_values, mode="lines", name=f"KDE {col}", showlegend=False
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=500,
    width=1200,
    title_text="General Parallel Metropolis Hastings: Parameters Overview",
)
fig.show()

In [5]:
fig = make_subplots(rows=2, cols=4)

for i, col in enumerate(samples.columns):
    row = (i // 4) + 1
    col_idx = (i % 4) + 1

    fig.add_trace(
        go.Box(
            y=samples[col],
            name=col,
            boxpoints="all",
            jitter=0.5,
            whiskerwidth=0.2,
            marker_size=2,
            line_width=1,
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=500,
    width=1200,
    title_text="General Parallel Metropolis Hastings: Boxplots of Parameters",
)
fig.show()

# Sampling

In [6]:
# Sampling Max
param_vec = []
for i in range(len(samples.loc[0])):
    values, counts = np.unique(samples.iloc[:, i], return_counts=True)
    ind = np.argmax(counts)
    param_vec.append(values[ind])

_, posterior_max, _, _ = run_model_single_parameter_node(model, param_vec)

[HVBSASK INFO] [0] parameters: [[3.2107858608107622, 2.967238586743689, 1.2001671033270265, 0.4355341822035047, 256.4992739700484, 0.22191583071346696, 0.04818322645776982]]


In [7]:
# Mean sampling from posterior
sample_param = []
for i in range(7):
    sample_param.append(np.random.choice(samples.iloc[:, i], 1000))
sample_param = np.array(sample_param).T

posterior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    posterior.append(y_model)

print(np.array(posterior).shape)
posterior_mean = np.mean(np.array(posterior), axis=0)

[HVBSASK INFO] [0] parameters: [array([3.01233892e+00, 3.24240447e+00, 1.21010463e+00, 3.24269638e-01,
       8.39893301e+01, 2.42008687e-01, 4.32395096e-02])]
[HVBSASK INFO] [0] parameters: [array([-1.26823696e+00,  2.91872370e+00,  2.19816248e+00,  4.10525746e-01,
        2.27038738e+02,  5.83677315e-01,  3.06143144e-02])]
[HVBSASK INFO] [0] parameters: [array([3.99495808e+00, 2.66388446e+00, 2.63285769e+00, 4.30123045e-02,
       2.21255332e+02, 1.79271338e-01, 2.53078176e-02])]
[HVBSASK INFO] [0] parameters: [array([-3.78854962e+00,  4.60689407e+00,  2.86694813e+00,  4.35534182e-01,
        4.93820071e+02,  2.21915831e-01,  4.65708695e-02])]
[HVBSASK INFO] [0] parameters: [array([1.96752633e+00, 5.88837111e-01, 2.19816248e+00, 6.82052086e-01,
       1.50986919e+02, 1.67258556e-01, 3.91267954e-02])]
[HVBSASK INFO] [0] parameters: [array([9.21004223e-01, 1.34895189e+00, 2.05749933e+00, 8.78741051e-01,
       1.69420011e+02, 4.57408304e-01, 3.43971144e-02])]
[HVBSASK INFO] [0] paramet

In [8]:
# Mean sampling from prior
sample_param = []
for i in range(7):
    uni = tfp.distributions.Uniform(low=param_lower[i], high=param_upper[i]).sample(1)
    sample_param.append(uni)
sample_param = np.array(sample_param).T

prior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    prior.append(y_model)

_, _, measured_data, _ = run_model_single_parameter_node(model, np.array(vec))

prior_means = np.mean(np.array(prior), axis=0)

[HVBSASK INFO] [0] parameters: [array([1.73690397e+00, 6.74880920e+00, 2.04553270e+00, 5.38687685e-01,
       4.04793783e+02, 7.34464554e-01, 4.49062722e-02])]
[HVBSASK INFO] [0] parameters: [array([1.73690397e+00, 6.74880920e+00, 2.04553270e+00, 5.38687685e-01,
       4.04793783e+02, 7.34464554e-01, 4.49062722e-02])]


2024-08-07 11:40:52.702517: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-08-07 11:40:52.702541: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-08-07 11:40:52.702548: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-08-07 11:40:52.702569: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-07 11:40:52.702583: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
fig = go.Figure()
model.get_start_date
dates = model.get_simulation_range()
fig.add_trace(
    go.Scatter(
        x=dates,
        y=prior_means,
        mode="lines",
        name="Prior Mean",
        line=dict(color="lightgrey"),
    )
)
fig.add_trace(
    go.Scatter(x=dates, y=posterior_mean, mode="lines", name="Posterior Mean")
)
fig.add_trace(go.Scatter(x=dates, y=posterior_max, mode="lines", name="Posterior Max"))
fig.add_trace(go.Scatter(x=dates, y=measured_data, mode="lines", name="Measured Data"))

# Update layout
fig.update_layout(
    title="General Parallel Metropolis Hastings: Bayesian Inference Result Comparison",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Time Series",
    hovermode="x unified",
    template="plotly_white",
)

fig.show()

In [10]:
def rmse(result, target):
    diff = result - target
    aggr = 0
    for i in range(len(diff)):
        aggr += diff[i] ** 2
    rmse = (aggr / (len(diff))) ** 0.5
    return rmse


def mae(result, target):
    return np.absolute(result - target).mean()


print(f"RMSE of Posterior Mean: {rmse(posterior_mean, measured_data)}")
print(f"RMSE of Posterior Max: {rmse(posterior_max, measured_data)}")
print(f"MAE of Posterior Mean: {mae(posterior_mean, measured_data)}")
print(f"MAE of Posterior Max: {mae(posterior_max, measured_data)}")

RMSE of Posterior Mean: 19.38010204522357
RMSE of Posterior Max: 21.53973595831511
MAE of Posterior Mean: 8.65291098967863
MAE of Posterior Max: 9.286754240054579
